In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as f
from pathlib import Path
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import os
from torchsummary import torchsummary
from PIL import Image

In [2]:
# Local

ORIGINAL_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'

In [ ]:
# On Server

ORIGINAL_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'

In [9]:
class NDIDatasetContrastiveLearning(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearning, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        origins, targets = [], []
        to_tensor_func = torchvision.transforms.ToTensor()
        for original_image in original_images:
            origins.append(to_tensor_func(Image.open(str(original_image))))
            targets.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))))
        random_index = np.random.permutation(len(origins))
        self.origins, self.targets = [], []
        for index in random_index:
            self.origins.append(origins[index])
            self.targets.append(targets[index])

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx]

    def __len__(self):
        return len(self.origins)

In [14]:
from moco_model import MoCo


model = MoCo(torchvision.models.resnet18, dim=128, K=48)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

In [11]:
# torchsummary.summary(model, input_size=[(1, 200, 200), (1, 200, 200)], batch_size=-1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [15]:
def train_moco(net, criterion, optimizer, epochs, device):
    train_iter = DataLoader(NDIDatasetContrastiveLearning(), batch_size=8, shuffle=True, drop_last=True)
    model.cuda(device)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for origin, target in train_iter:
            origin, target = origin.cuda(device), target.cuda(device)
            output, labels = net(origin, target)
            loss = criterion(output, labels)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss {total_loss / len(train_iter)}')

In [16]:
train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 2.624095919820926
Epoch 2, Loss 3.8455854256947837
Epoch 3, Loss 3.729530930519104
Epoch 4, Loss 3.5079191525777182
Epoch 5, Loss 3.190559903780619
Epoch 6, Loss 2.9634703397750854
Epoch 7, Loss 3.1603736877441406
Epoch 8, Loss 2.9089460372924805
Epoch 9, Loss 2.9818639755249023
Epoch 10, Loss 2.76382847627004
Epoch 11, Loss 2.625672698020935
Epoch 12, Loss 2.5617246627807617
Epoch 13, Loss 2.5302723248799643
Epoch 14, Loss 2.4561252991358438
Epoch 15, Loss 2.4001572330792746
Epoch 16, Loss 2.316851099332174
Epoch 17, Loss 2.1289164423942566
Epoch 18, Loss 2.18250838915507
Epoch 19, Loss 2.172099471092224
Epoch 20, Loss 2.0479262868563333
Epoch 21, Loss 1.8936439553896587
Epoch 22, Loss 1.9546444217363994
Epoch 23, Loss 1.8207480510075886
Epoch 24, Loss 1.7978564302126567
Epoch 25, Loss 1.725442349910736
Epoch 26, Loss 1.6509392460187275
Epoch 27, Loss 1.6228861808776855
Epoch 28, Loss 1.6434180339177449
Epoch 29, Loss 1.5815683404604595
Epoch 30, Loss 1.5814982851346333


In [17]:
def image_pair_matching(net, original_image, matching_image):
    net.cpu()
    net.eval()
    q = net.encoder_q(original_image)
    q = f.normalize(q, dim=1)
    k = net.encoder_k(matching_image)
    k = f.normalize(k, dim=1)
    logits = torch.mm(q, k.T)
    return logits

In [18]:
# Try to take the size of queue less than K, success

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_images_candidates = np.random.choice(target_images, 40)
target_tensor = []
for i in range(20):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))).unsqueeze(0))
for i in range(20, 40):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
image_pair_matching(model, original_tensor, target_tensor)

tensor([[ 0.0712, -0.0359, -0.3427, -0.2899, -0.0107, -0.0201,  0.1613,  0.0990,
          0.0682,  0.1006,  0.1205, -0.3328,  0.2456,  0.0248,  0.1088,  0.2851,
          0.2203,  0.0070,  0.3048,  0.0915,  0.3757,  0.0682,  0.1205, -0.2899,
          0.0070,  0.3757, -0.0160, -0.0243, -0.0359, -0.2899,  0.2456,  0.0237,
          0.0947, -0.3013, -0.0243, -0.0294,  0.0712,  0.0553,  0.0682,  0.1006,
          0.1006]], grad_fn=<MmBackward>)

In [19]:
# Try to take all matching images

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_tensor = []
for i in range(1, 55):
    target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
print(str(original_image.name).split('_')[0])

6


In [20]:
logits = image_pair_matching(model, original_tensor, target_tensor)
print(logits)
print(torch.argmax(logits))

tensor([[0.0627, 0.0611, 0.0699, 0.0440, 0.2474, 0.4103, 0.1317, 0.1803, 0.2568,
         0.0357, 0.0792, 0.1102, 0.2470, 0.1646, 0.0851, 0.0911, 0.0696, 0.1060,
         0.2821, 0.0943, 0.1490, 0.0588, 0.0896, 0.2587, 0.1505, 0.1505, 0.0533,
         0.0516, 0.1382, 0.1897, 0.0510, 0.0545, 0.1133, 0.1614, 0.0665, 0.1281,
         0.2796, 0.2284, 0.1127, 0.1159, 0.0888, 0.1651, 0.1685, 0.0181, 0.0515,
         0.1277, 0.0485, 0.1728, 0.0496, 0.0112, 0.1372, 0.0910, 0.0910, 0.1739]],
       grad_fn=<MmBackward>)
tensor(5)
